In [ ]:
%matplotlib auto
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import sys
import os.path
sys.path.insert(0, os.getcwd()+"/../../../libs/tacx")
from SEMContour import SEMContour
sys.path.insert(0, os.getcwd()+"/../../../libs/common")
from FileUtil import gpfs2WinPath

CWD = '/gpfs/WW/BD/MXP/SHARED/SEM_IMAGE/IMEC/Case02_calaveras_v3/3Tmp/CT_KPI_test/Calaveras_v3_regular_CT_KPI_003_slope_modified_revert_all_patterns/h/cache/dummydb/result/MXP/job1/ContourSelectModelCalibration430result1'
#CWD = r'C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\samplejob\h\cache\dummydb\result\MXP\job1\ContourSelectModelCalibration430result1'
#CWD = r'C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\samplejob1\h\cache\dummydb\result\MXP\job1\ContourExtraction400result1'
CWD = gpfs2WinPath(CWD)
allNeighborColNames = ['NeighborContinuity', 'NeighborOrientation', 'NeighborParalism']

class ContourAnalyzer(object):
    """docstring for ContourData"""
    def __init__(self, contourfile):
        self.__build(contourfile)

    def __build(self, contourfile):
        contour = SEMContour()
        contour.parseFile(contourfile)
        if not contour:
            sys.exit("ERROR: read in cmntour file %s fails\n" % contourfile)
        self.contour = contour
        self.df = contour.toDf()
# get contour data
patternid = '2438'
contourfile = os.path.join(CWD, patternid+'_image_contour.txt')
ca = ContourAnalyzer(contourfile)
df = ca.df

In [ ]:
# plot by column unique labels
def plot_col_by_label(ca, patternid='', colname=''):
    contour = ca.contour
    df = ca.df
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    xini, yini, xend, yend = contour.getBBox()
    ax.set_xlim([xini, xend])
    ax.set_ylim([yini, yend])
    ax.set_title("Pattern "+patternid)
    
    uniqVals = df.loc[:, colname].drop_duplicates().values
    print(uniqVals)
    for label in uniqVals:
        flt_eq = df.loc[:, colname] == label
        if label == 'nan':
            flt_eq = df.loc[:, colname].isna()
        ax.plot(df.loc[flt_eq, 'offsetx'], df.loc[flt_eq, 'offsety'], '.', linestyle='None',  markersize=2, label=colname+'=={}'.format(label))

    plt.gca().invert_yaxis()
    plt.legend()
    plt.show()

In [ ]:
def addNeighborFeatures(df):
    '''
    add Features for the input contour DataFrame, based on the neighbor relationship in the context of segment

    Parameters:
    -----------
    df: [in, out] contour as DataFrame
        [in] Contour df, must contains `polygonId`, `angle`, `offsetx`, `offsety`
        [out] Contour df, added `NeighborContinuity`, `NeighborOrientation`, `NeighborParalism`

            - `NeighborContinuity`:  |X(n) - X(n-1)|^2, usually is to 1 (because of 8-neighbor contour tracing)
            - `NeighborOrientation`:  dot(EigenVector(n), EigenVector(n-1)), closer to 1, the better(may use 1-dot)
            - `NeighborParalism`:  ||cross((X(n) - X(n-1)), EigenVector(n-1))||, closer to 1, the better(may use 1-cross)
    Note, the segment neighborhood based features can only be obtained by the whole segment, can't use ROI cropped segment 
    '''
    if len(df) <= 0:
        return df
    polygonIds = df.loc[:, 'polygonId'].drop_duplicates().values
    preIdx = df.index[0]
    for polygonId in polygonIds:
        # search forward 
        neighborFeatureValForward = []
        isPolygonHead = True
        for curIdx in df.loc[df['polygonId']==polygonId, :].index:
            NeighborContinuity = 1
            NeighborOrientation = 1
            NeighborParalism = 1
            if not isPolygonHead:
                eigenvector_n_1 = np.array([np.cos(df.loc[preIdx, 'angle']), np.sin(df.loc[preIdx, 'angle'])])
                eigenvector_n = np.array([np.cos(df.loc[curIdx, 'angle']), np.sin(df.loc[curIdx, 'angle'])])
                neighorvector = np.array([df.loc[curIdx, 'offsetx'] - df.loc[preIdx, 'offsetx'],
                                        df.loc[curIdx, 'offsety'] - df.loc[preIdx, 'offsety']])
                crossvector = np.cross(neighorvector, eigenvector_n_1)

                NeighborContinuity = np.sqrt(neighorvector.dot(neighorvector))
                NeighborOrientation = eigenvector_n.dot(eigenvector_n_1)
                NeighborParalism = np.sqrt(crossvector.dot(crossvector))/NeighborContinuity
                NeighborContinuity = NeighborContinuity
            preIdx = curIdx
            isPolygonHead = False
            neighborFeatureValForward.append((NeighborContinuity, NeighborOrientation, NeighborParalism))

        # search backward
        neighborFeatureValBackward = []
        isPolygonHead = True
        for curIdx in df.loc[df['polygonId']==polygonId, :].index[::-1]:
            NeighborContinuity = 1
            NeighborOrientation = 1
            NeighborParalism = 1
            if not isPolygonHead:
                eigenvector_n_1 = np.array([np.cos(df.loc[preIdx, 'angle']), np.sin(df.loc[preIdx, 'angle'])])
                eigenvector_n = np.array([np.cos(df.loc[curIdx, 'angle']), np.sin(df.loc[curIdx, 'angle'])])
                neighorvector = np.array([df.loc[curIdx, 'offsetx'] - df.loc[preIdx, 'offsetx'],
                                        df.loc[curIdx, 'offsety'] - df.loc[preIdx, 'offsety']])
                crossvector = np.cross(neighorvector, eigenvector_n_1)

                NeighborContinuity = np.sqrt(neighorvector.dot(neighorvector))
                NeighborOrientation = eigenvector_n.dot(eigenvector_n_1)
                NeighborParalism = np.sqrt(crossvector.dot(crossvector))/NeighborContinuity
                NeighborContinuity = NeighborContinuity
            preIdx = curIdx
            isPolygonHead = False
            neighborFeatureValBackward.append((NeighborContinuity, NeighborOrientation, NeighborParalism))

        # merge results
        neighborFeatureValBackward = neighborFeatureValBackward[::-1]
        for ii, curIdx in enumerate(df.loc[df['polygonId']==polygonId, :].index):
            for jj, colname in enumerate(allNeighborColNames):
                '''
                if ii == 0 or ii == len(neighborFeatureValForward)-1:
                    if abs(neighborFeatureValForward[ii][jj] - 1) >= abs(neighborFeatureValBackward[ii][jj] - 1):
                        val = neighborFeatureValForward[ii][jj]
                    else:
                        val = neighborFeatureValBackward[ii][jj]
                else:
                    val = 0.5*(neighborFeatureValForward[ii][jj] + neighborFeatureValBackward[ii][jj])
                '''
                if abs(neighborFeatureValForward[ii][jj] - 1) >= abs(neighborFeatureValBackward[ii][jj] - 1):
                    val = neighborFeatureValForward[ii][jj]
                else:
                    val = neighborFeatureValBackward[ii][jj]
                df.loc[curIdx, colname] = val
    return df

def plot_multi_filters(ca, patternid='', strFlts=None, transform_filter=False):
    if strFlts is None:
        newStrFlts = {}
    elif not isinstance(strFlts, dict):
        newStrFlts = {}
        for col in allNeighborColNames:
            for strFlt in strFlts:
                if col in strFlt:
                    newStrFlts[col] = strFlt
                    break
    strFlts = newStrFlts
    print(strFlts)
    df = ca.df
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    xini, yini, xend, yend = ca.contour.getBBox()
    ax.set_xlim([xini, xend])
    ax.set_ylim([yini, yend])
    ax.set_title("Pattern "+patternid)
    
    # plot contour
    ax.plot(df.loc[:, 'offsetx'], df.loc[:, 'offsety'], 'k.', markersize=1, label='SEM Contour')
    # plot angle
    for _, row in df.iterrows():
        x, y = row.loc['offsetx'], row.loc['offsety']
        angle = row.loc['angle']
        arrow_length = 1
        dx, dy = arrow_length*np.cos(angle), arrow_length*np.sin(angle)
        ax.arrow(x, y, dx, dy, width=0.1, fc='y', ec='y') # ,shape='right', overhang=0

    # plot filters
    if not transform_filter:
        for strFlt in strFlts.values():
            curdf = df.query(strFlt)
            ax.plot(curdf.loc[:, 'offsetx'], curdf.loc[:, 'offsety'], 'o', markersize=4, label=strFlt, alpha=0.6)
    else:
        '''
        inflection_points = []
        for strFlt in strFlts.values():
            curdf = df.query(strFlt )
            print(strFlt, len(curdf))
            if len(curdf) != 0:
                inflection_points.append(curdf)
        inflection_df = pd.concat(inflection_points)
        '''
        inflection_df = df.query('or '.join(strFlts.values()))
        print(inflection_df[['polygonId', 'offsetx', 'offsety'] + allNeighborColNames[1:]])
        
        polygonIds = inflection_df.loc[:, 'polygonId'].drop_duplicates().values
        for polygonId in polygonIds:
            curdf = inflection_df.loc[inflection_df['polygonId']==polygonId, :]
            maxNeighborContinuity = curdf.max()['NeighborContinuity']
            minNeighborContinuity, minNeighborOrientation, minNeighborParalism = curdf.min()[allNeighborColNames]

            if minNeighborParalism < minNeighborOrientation and len(curdf.query(strFlts['NeighborParalism'])) > 0:
                idxmin = curdf['NeighborParalism'].idxmin()
                ax.plot(curdf.loc[idxmin, 'offsetx'], curdf.loc[idxmin, 'offsety'], 'ro', markersize=4, label='minNeighborParalism', alpha=0.6)
            elif minNeighborOrientation < minNeighborParalism and len(curdf.query(strFlts['NeighborOrientation'])) > 0:
                idxmin = curdf['NeighborOrientation'].idxmin()
                ax.plot(curdf.loc[idxmin, 'offsetx'], curdf.loc[idxmin, 'offsety'], 'rd', markersize=4, label='minNeighborOrientation', alpha=0.6)
            elif len(curdf.query(strFlts['NeighborContinuity'])) > 0:
                if abs(maxNeighborContinuity-1) > abs(minNeighborContinuity-1):
                    idxmax = curdf['NeighborContinuity'].idxmax()
                    ax.plot(curdf.loc[idxmax, 'offsetx'], curdf.loc[idxmax, 'offsety'], 'bd', markersize=4, label='maxNeighborContinuity', alpha=0.6)
                else:
                    idxmin = curdf['NeighborContinuity'].idxmin()
                    ax.plot(curdf.loc[idxmin, 'offsetx'], curdf.loc[idxmin, 'offsety'], 'bo', markersize=4, label='minNeighborContinuity', alpha=0.6)

    plt.gca().invert_yaxis()
    plt.legend()
    plt.show()
    return inflection_df
    
    #inflection_df.plot.scatter(x=allNeighborColNames[1], y=allNeighborColNames[2])
    #plt.show()
#inflection_df = plot_multi_filters(ca, patternid=patternid, strFlts=['abs(1-NeighborContinuity) > 0.5', 'NeighborParalism<0.98', 'NeighborOrientation<0.98'], transform_filter=True)
df = addNeighborFeatures(df)
ca.df = df
inflection_df = plot_multi_filters(ca, patternid=patternid, strFlts=['abs(1-NeighborContinuity) > 0.5', 'NeighborParalism<0.98', 'NeighborOrientation<0.98'], transform_filter=True)

In [ ]:
def calcMeanOfLargestHistBin(arr, bins=10):
    hist, bin_edges = np.histogram(arr, bins=bins)
    idxmax = np.argmax(hist)
    binvals = arr[np.where(np.logical_and(arr>=bin_edges[idxmax], arr<bin_edges[idxmax+1]))]
    return np.mean(binvals)

def findIndexOfFirstFlat(arr, gradients=None, start_pos=0, thres=None):
    if gradients is None:
        gradients = np.gradient(arr, edge_order=2)
    assert(len(arr) == len(gradients))
    absGradients = np.abs(gradients)
    if thres is None:
        thres = calcMeanOfLargestHistBin(absGradients)
    for ix in range(start_pos, len(gradients)):
        if absGradients[ix] < thres:
            print(thres, ix)
            return ix
    return start_pos

def findIndexOfFirstZeroCrossing(arr, gradients=None, start_pos=0):
    if gradients is None:
        gradients = np.gradient(arr, edge_order=2)
    assert(len(arr) == len(gradients))
    start = start_pos+1 if start_pos == 0 else start_pos
    for ix in range(start_pos, len(gradients)):
        if gradients[ix]*gradients[ix-1] <=0 :
            return ix
    return start_pos


flt = [0.10650698, 0.78698604, 0.10650698] # sigma = 0.5
#flt = [0.17, 0.66, 0.17] # sigma 0.6
#flt = [0.25, 0.5, 0.25] # sigma 0.8
print(flt)
def smoothSignal(arr):
    padArr = np.zeros((arr.shape[0]+2,) )
    padArr[0] = arr[0]
    padArr[1:-1] = arr
    padArr[-1] = arr[-1]
    '''
    arr = list(arr)
    padArr = arr[0:1] + arr[:] + arr[-1:]
    '''
    newArr = np.convolve(padArr, flt, 'valid')
    if len(arr) != len(newArr):
        raise ValueError("inequal length {} {}".format(len(arr), len(newArr)))
    return newArr

#vlines = inflection_df.groupby(['polygonId'])
for polygonId in [29]: 
    print(polygonId)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    curdf = df.loc[df.polygonId==polygonId, :]
    x = np.arange(len(curdf))
    arr1 = curdf[allNeighborColNames[1]].values
    arr2 = curdf[allNeighborColNames[2]].values
    #arr2 = smoothSignal(arr2)
    gradient1 = np.gradient(arr1, edge_order=2)
    gradient2 = np.gradient(arr2, edge_order=2)
    Paralism_xx = np.gradient(gradient2, edge_order=2)
    
    idxmin = np.argmin(arr2)
    start_pos = min(idxmin+1, len(arr2)-1)
    ax.plot(idxmin, arr2[idxmin], 'o', markersize=12, markeredgewidth=2, markerfacecolor='none', label= 'min'+allNeighborColNames[2])
    idx1 = findIndexOfFirstFlat(arr2, gradient2, start_pos=start_pos)
    idx2 = findIndexOfFirstZeroCrossing(arr2, gradient2, start_pos=start_pos)
    ax.plot(idx1, arr2[idx1], 'o', markersize=12, markeredgewidth=2, markerfacecolor='none', label= allNeighborColNames[2] + ' zero gradient')
    ax.plot(idx2, arr2[idx2], 'o', markersize=12, markeredgewidth=2, markerfacecolor='none', label= allNeighborColNames[2] + ' zero-crossing gradient')
    
    
    #ax.plot(x, (curdf[allNeighborColNames[0]]-1).abs(), 'g-d', label=allNeighborColNames[0])
    #ax.plot(x, curdf[allNeighborColNames[1]], '-o', label= allNeighborColNames[1])
    #ax.plot(x, curdf[allNeighborColNames[2]], '--o', label= allNeighborColNames[2], alpha=0.6)
    ax.plot(x, arr2, '--o', label= allNeighborColNames[2], alpha=0.6)
    #ax.plot(x, gradient1, '-*', label= allNeighborColNames[1] + ' gradient')
    #ax.plot(x, gradient2, '--*', label= allNeighborColNames[2] + ' gradient')
    #ax.plot(x, Paralism_xx, '--d', label= allNeighborColNames[2] + ' 2nd deriative')
    
    if idxmin in range(40):
        ax.set_xlim([0, 40])
    #plt.yscale('log')    
    ax.set_title("Pattern {} Vline {}".format(patternid, polygonId))
    plt.legend()

In [ ]:
outColName='ClfLabel'

def categorizeFilters(filters):
    if filters is None:
        newfilters = {}
    elif not isinstance(filters, dict):
        newfilters = {}
        for col in allNeighborColNames:
            for strFlt in filters:
                if col in strFlt:
                    newfilters[col] = strFlt
                    break
    filters = newfilters
    # print(filters)
    return filters

def applyNeighborRuleModelPerVLine(linedf, filters=None, maxTailLength=20, smooth=True):
    dominant_issues = []
    linedf.loc[:, outColName] = 1

    # step 1, search and apply from head
    headLength = min(len(linedf), maxTailLength)
    headdf = linedf.loc[linedf.index[:headLength], :]
    minNeighborOrientation, minNeighborParalism = headdf.min()[allNeighborColNames[1:]]
    issue_feature, issue_index = None, None
    if minNeighborParalism < minNeighborOrientation and len(headdf.query(filters['NeighborParalism'])) > 0:
        issue_feature = 'NeighborParalism'
        issue_index = np.argmin(headdf[issue_feature].values)
    elif minNeighborOrientation < minNeighborParalism and len(headdf.query(filters['NeighborOrientation'])) > 0:
        issue_feature = 'NeighborOrientation'
        issue_index = np.argmin(headdf[issue_feature].values)
    dominant_issues.append(None)
    if issue_feature is not None:
        dominant_issues[0] = [issue_feature, issue_index]
        arr = linedf.loc[:, issue_feature].values
        if smooth:
            arr = smoothSignal(arr)
        gradient = np.gradient(arr, edge_order=2)
        start_pos = min(issue_index+1, len(arr)-1)
        idxFlat = findIndexOfFirstFlat(arr, gradient, start_pos=start_pos)
        dominant_issues[0].append(idxFlat)
        linedf.loc[linedf.index[:idxFlat], outColName] = 0

    # step 2, search and apply from tail, reverse order
    dominant_issues.append(None)
    head_index = 0 if issue_index is None else issue_index
    tailrange = len(linedf) - (head_index + 1) # exclude the head issue index itself
    if tailrange > 0:
        tailLength = min(tailrange, maxTailLength)
        taildf = linedf.loc[linedf.index[-tailLength:], :]
        minNeighborOrientation, minNeighborParalism = taildf.min()[allNeighborColNames[1:]]
        issue_feature, issue_index = None, None
        if minNeighborParalism < minNeighborOrientation and len(taildf.query(filters['NeighborParalism'])) > 0:
            issue_feature = 'NeighborParalism'
            issue_index = np.argmin(taildf[issue_feature].values)
            issue_index = tailLength - 1 - issue_index  # use index start from tail
        elif minNeighborOrientation < minNeighborParalism and len(taildf.query(filters['NeighborOrientation'])) > 0:
            issue_feature = 'NeighborOrientation'
            issue_index = np.argmin(taildf[issue_feature].values)
            issue_index = tailLength - 1 - issue_index
        if issue_feature is not None:
            dominant_issues[1] = [issue_feature, issue_index]
            arr = linedf.loc[:, issue_feature].values[::-1]
            if smooth:
                arr = smoothSignal(arr)
            gradient = np.gradient(arr, edge_order=2)
            start_pos = min(issue_index+1, len(arr)-1)
            idxFlat = findIndexOfFirstFlat(arr, gradient, start_pos=start_pos)
            dominant_issues[1].append(idxFlat)
            linedf.loc[linedf.index[-idxFlat:], outColName] = 0
    return linedf, dominant_issues

def applyNeighborRuleModel(contourdf, filters, smooth=True):
    '''
    The step to find rule model in python:
    1. apply combined filters to find ill contour Vline candidates
    2. find the Vline candidates have dominant issues in its head+20 or tail-20
    3. remove contour issue head/tail by following rules(default is 3.1):
        * 3.1: search start from dominant issue position, new head=Index[1st flat gradient point]
        * 3.2: Index[dominant issue position], new head=Index[the gradient zero-crossing point]
    '''
    maxTailLenth = 20
    contourdf.loc[:, 'ClfLabel'] = 1
    filters = categorizeFilters(filters)

    inflection_df = contourdf.query('or '.join(filters.values()))
    # print(inflection_df[['polygonId', 'offsetx', 'offsety'] + allNeighborColNames[1:]])
    polygonIds = inflection_df.loc[:, 'polygonId'].drop_duplicates().values
    for polygonId in polygonIds:
        lineFlt = contourdf['polygonId']==polygonId
        linedf = contourdf.loc[lineFlt, :]
        newlinedf, dominant_issues = applyNeighborRuleModelPerVLine(linedf, filters, maxTailLenth, smooth)
        print(int(polygonId), dominant_issues)
        contourdf.loc[lineFlt, :] = newlinedf
    return contourdf

df = applyNeighborRuleModel(df, ['NeighborOrientation<0.98', 'NeighborParalism<0.98'])
ca.df = df
plot_col_by_label(ca, patternid=patternid, colname='ClfLabel')

In [ ]:
# SEM Contour Selection resulst plot: by classifer Positive 0, & Negative 1
def plotContourDiscriminator(contour, im=None, fig=None, subidx=111, wndname=''):
    # plot image and classified contour point
    df = contour.toDf()
    if any([col not in df.columns for col in ['ClfLabel']]):
        return False

    # fig = plt.figure()
    if subidx >= 111:
        ax = fig.add_subplot(subidx)
    else:
        ax = fig.add_subplot(2, 4, subidx)
    
    imw, imh = contour.getshape()
    '''
    ax.set_aspect('equal')
    ax.set_xlim([0, imw])
    ax.set_ylim([0, imh])
    '''
    xini, yini, xend, yend = contour.getBBox()
    ax.set_xlim([xini, xend])
    ax.set_ylim([yini, yend])
    ax.set_title(wndname)
    
    Positive = df.ClfLabel==0
    Negative = df.ClfLabel==1

    # calculate confusion matrix
    cm = np.array([len(df.loc[flt, :]) for flt in [Positive, Negative]])
    cm_norm = cm.astype('float') / cm.sum()
    
    if im is not None:
        if len(im.shape) == 2:
            im = cv2.cvtColor((im/65536.).astype('float32'), cv2.COLOR_GRAY2BGR)
        ax.imshow(im)
    ax.plot(df.loc[Positive ,'offsetx'], df.loc[Positive, 'offsety'], #'b.', markersize=1, 
        linestyle='None', marker= 'o', markeredgecolor='r', markersize=2, markeredgewidth=1, markerfacecolor='none',
        label='remove: {}({:.3f}%)'.format(cm[0], cm_norm[0]*100 )) #Discriminator Positive, ClfLabel=0
    ax.plot(df.loc[Negative ,'offsetx'], df.loc[Negative, 'offsety'], #'r*', markersize=2,
        linestyle='None', marker= '.', markeredgecolor='b', markersize=2, markeredgewidth=1, markerfacecolor='none', 
        label='Keep: {}({:.3f}%)'.format(cm[1], cm_norm[1]*100 )) #Discriminator Negative, ClfLabel=1:
    
    #ax = plt.gca() # gca() function returns the current Axes instance
    #ax.set_ylim(ax.get_ylim()[::-1]) # reverse Y
    plt.gca().invert_yaxis()
    plt.legend(loc=1)
    plt.show()
    return True

def plotComparedContourResults():

    sys.path.insert(0, os.getcwd()+"/../../../libs/tacx")
    print(os.getcwd()+"/../../../libs/tacx")
    sys.path.insert(0, os.getcwd()+"/../../../libs/common")
    
    jobpath = '/gpfs/WW/BD/MXP/SHARED/SEM_IMAGE/IMEC/Case02_calaveras_v3/3Tmp/CT_KPI_test/Calaveras_v3_regular_CT_KPI_003_slope_modified_revert_all_patterns/'
    resultpaths = [
          'h/cache/dummydb/result/MXP/job1/ContourSelectModelCalibration430result1', # clf 
          'h/cache/dummydb/result/MXP/job1/ContourSelectModelCalibration431result1', # rule
          ]
    impath = 'h/cache/dummydb/result/MXP/job1/Average301result1'
    modeltypes = ['clf', 'rule']
    CWDs = [''.join([jobpath, path]) for path in resultpaths]
    impath = ''.join([jobpath, impath])
    
    ''' # comment block 1 starts
    #################
    # type 1, review model apply image by random permutation
    #################
    pathfilter = '*_image_contour.txt'
    pathex = gpfs2WinPath(os.path.join(CWD, pathfilter))
    contourfiles = glob.glob(pathex)
    contourindice = np.random.permutation(np.arange(len(contourfiles)))
    for ii in range(0*8, 1*8):
        fig = plt.figure()
        for jj, idx in enumerate(contourindice[ii*8:(ii+1)*8]):
            contourfile = contourfiles[idx]
            patternid = os.path.basename(contourfile).strip('_image_contour.txt')
            ################# end of type 1
    ''' # comment block 1 ends
            
    #################
    # type 2, review model apply image by giving list
    #################
    #patternids = [461, 1001]
    #patternids = [118, 430, 432, 437, 442, 449, 461, 530, 536, 539, 542, 833, 1593, 1785, 1793, 1801, 1809, 2163, 2196, 2284, 2405, 2427, 2438, 2543, 2585, 2655, 2697, 2767, 3223, 3418, 3463, 3553, 3583, 3613, 3628]
    #patternids = [432, 442, 449, 536, 1785, 2405, 2427, 2438,  2767, 2697, 3418, 3583, 3613, 49, 63, 438, 439, 3741,  3742]
    patternids = [449, 1785, 2405, 2438,2767, 3418, 3583]

    
    psteps = 4 # pattern steps in the same outxml file
    for ii in range(int(np.ceil(len(patternids)/float(psteps)))):
        fig = plt.figure()
        for jj, idx in enumerate(range(ii*psteps, (ii+1)*psteps)):
            if idx > len(patternids):
                return
            for kk, CWD in enumerate(CWDs):
                try:
                    patternid = str(patternids[idx])
                except IndexError:
                    raise IndexError(patternid)
                contourfile = gpfs2WinPath(os.path.join(CWD, patternid+'_image_contour.txt'))
                ################# end of type 2        
                
                if not os.path.exists(contourfile):
                    print(patternid+' not exist')
                    continue
        
                # get contour data
                contour = SEMContour()
                if not contour.parseFile(contourfile):
                    continue
                if kk == 1:
                    df = contour.toDf()
                    df = addNeighborFeatures(df)
                    df = applyNeighborRuleModel(df, ['NeighborOrientation<0.98', 'NeighborParalism<0.98'])
                    contour = contour.fromDf(df)
                #im, _ = imread_gray(os.path.join(impath, patternid+'_image.pgm'))
                
                plotContourDiscriminator(contour, fig=fig, subidx=kk*psteps+jj+1, wndname=modeltypes[kk]+' model Pattern '+ patternid)
        
plotComparedContourResults()